# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 2

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [97]:
import numpy as np
import operator
import collections
import pandas
from random import randrange
from sklearn.metrics import confusion_matrix as cm
#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score

### Leitura da base "demartology"

In [98]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       366 non-null    int64 
 1   1       366 non-null    int64 
 2   2       366 non-null    int64 
 3   3       366 non-null    int64 
 4   4       366 non-null    int64 
 5   5       366 non-null    int64 
 6   6       366 non-null    int64 
 7   7       366 non-null    int64 
 8   8       366 non-null    int64 
 9   9       366 non-null    int64 
 10  10      366 non-null    int64 
 11  11      366 non-null    int64 
 12  12      366 non-null    int64 
 13  13      366 non-null    int64 
 14  14      366 non-null    int64 
 15  15      366 non-null    int64 
 16  16      366 non-null    int64 
 17  17      366 non-null    int64 
 18  18      366 non-null    int64 
 19  19      366 non-null    int64 
 20  20      366 non-null    int64 
 21  21      366 non-null    int64 
 22  22      366 non-null    in

#### Pre-processamento

Olhando o conteúdo do arquivo, foi notado que algumas linhas da coluna 33 (band-like infiltrate) apresentam valores marcados com "?" que atrapalham o resultado do algoritmo. Para remediar esse problema , decidi trocar os valores faltantes demarcados por "?" pela mediana da coluna 33, pois tal medida é menos sensível a outliers se comparado com a média, por exemplo.

In [99]:
def calc_median(array):
    values = []
    for v in array:
        if v != '?':
            v = int(v)
            values.append(v)
    return np.median(values)

median = calc_median(data.iloc[:,33])
data.iloc[:,33] = list(map(lambda value: median if value == '?' else value, data.iloc[:,33]))
data.iloc[:,33] = data.iloc[:,33].astype(np.int64)

### Principal Component Analysis

Abaixo temos os passos necessários para calcular o PCA.

<ol>
<li>Calcular médias de todas as features e subtrair as mesmas de cada feature</li>
<li>Calcular matriz de covariância com base nos dados modificados pelo passo 1</li>
<li>Calcular autovetores e autovalores</li>
<li>Escolher o número Q que representa a quantidade de autovetores</li>
<li>Multiplicar os autovetores escolhidos pela matriz resultante do passo 1</li>
</ol>

#### Passo 1 -  Calcular médias de todas as features e subtrair as mesmas de cada feature

In [1]:
def remove_median(dataframe):
    columns = data.columns.values
    for c in columns:
        mean = data[c].mean()
        data[c] = data[c].apply(lambda value : value - mean)
    return dataframe

#### Passo 2 - Calcular matriz de covariância com base nos dados modificados pelo passo 1

In [2]:
def calc_cov(dataframe):
    return np.cov(dataframe, rowvar = False)

#### Passo 3 - Calcular autovetores e autovalores

In [6]:
def calc_eigen_values_and_vectors(data):
    eigen_values , eigen_vectors = np.linalg.eigh(data)
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
    return sorted_eigenvalue,sorted_eigenvectors

#### Passo 4 - Escolher o número Q que representa a quantidade de autovetores

In [4]:
def chooseQ(eigen_values,eigen_vectors,percentual):
    total = 0
    Q = 0
    for index in range(len(eigen_values)):
        total += eigen_values[index]/np.sum(eigen_values)
        Q += 1
        if total >= percentual:
            break
    return Q

#### Passo 5 - Multiplicar os autovetores escolhidos pela matriz resultante do passo 1

In [5]:
def pca(dataframe, eigen_vectors):
    return np.dot(eigen_vectors.transpose() , dataframe.transpose() ).transpose()

In [8]:
## eigen_values , eigen_vectors = calc_eigen_values_and_vectors(data)
## q = chooseQ(eigen_values , eigen_vectors, 0.95)
## print(eigen_vectors[:q,])
## print(pca(data,eigen_vectors[:,:q]))